In [ ]:
import pandas as pd
import numpy as np
import openai

from sklearn.mixture import GaussianMixture
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from konlpy.tag import Mecab
from umap import UMAP

from model.model_utils import *
from crawler.getter import *

/home/changoo/.local/share/virtualenvs/comment-6N9L2wjh/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## CommentEmbedder

- get_comments: youtube data api로 댓글을 가져와 xlsx에 저장하고 읽어옵니다.
- eliminate_stopwords: 불용어를 제거합니다.
- get_embeddings: 각 댓글의 embedding과 단어들의 embedding을 연산합니다.

In [ ]:
class CommentEmbedder:
    def __init__(self, model, mecab, stopwords):
        self.df = None
        self.mecab = mecab
        self.model = model
        self.stopwords = stopwords
    
    def get_comments(self, video_id):
        filename = get_comments_by_video_ids(video_id)
        self.df = read_excel(filename)

    def eliminate_stopwords(self):
        comments = self.df['comment'].to_list()
        self.df = self.df.rename(columns={'comment': 'raw_comment'})
        self.df['comment'] = [extract_nouns(comment, self.mecab, self.stopwords) for comment in comments]

    def get_embbedings(self):
        # 댓글별 임베딩
        comments = self.df['comment'].to_list()
        self.df['embeddings'] = self.model.encode(comments, batch_size=32).tolist()

        # 단어별 임베딩
        tokenized_comments = []
        unique_words = set()
        for comment in comments:
            try:
                words = CountVectorizer(ngram_range=(1, 1)).fit([comment]).get_feature_names_out()
            except ValueError:
                words = []
            tokenized_comments.append(words)
            unique_words.update(words)
        self.df['words'] = tokenized_comments
        
        unique_word_list = list(unique_words)
        word_embeddings = self.model.encode(unique_word_list, batch_size=32)
        word_embeddings_dict = dict(zip(unique_word_list, word_embeddings))
        
        return self.df, word_embeddings_dict


## KeywordExtractor

- get_keyword_groups: 키워드 그룹들에 댓글들을 할당합니다.
- extract_keyword_groups: 중복댓글을 처리하고 TOP-N 그룹만 추출하여 중요한 댓글만 뽑아냅니다.

In [94]:
class KeywordExtractor:
    def __init__(self, df, word_embeddings_dict):
        self.df = df
        self.word_embeddings_dict = word_embeddings_dict
        self.sorted_keywords = None

    def extract_keywords(self, doc_embedding, words, word_embeddings_dict, top_n=5):
        if len(words) == 0:
            return []
        word_embedding = [word_embeddings_dict[word] for word in words]

        # Cosine-based keyword extraction
        distances = cosine_similarity([doc_embedding], word_embedding)
        keywords = [
            # (words[index], round(float(distances[0][index]), 4))
            words[index]
            for index in distances.argsort()[0][-top_n:]
        ][::-1]

        return keywords

    def get_keyword_groups(self):
        doc_embeddings = self.df['embeddings'].to_list()
        tokenized_sentences = self.df['words'].to_list()
        keywords_per_comment = [
            self.extract_keywords(doc_embedding, words, self.word_embeddings_dict, top_n=2) 
            for doc_embedding, words in zip(doc_embeddings, tokenized_sentences)
        ]

        keyword_to_comments = defaultdict(list)
        for comment_idx, keywords in enumerate(keywords_per_comment):
            for keyword in keywords:
                keyword_to_comments[keyword].append(comment_idx)

        self.sorted_keywords = sorted(
            keyword_to_comments.items(),
            key=lambda x: len(x[1]),
            reverse=True
        )
        return self.sorted_keywords

    def extract_keyword_groups(self):
        # 키워드 합치기 & 중복 제거
        indices = {keyword: set(index_list) for keyword, index_list in self.sorted_keywords}
        # 댓글 수 Threshold
        overlap_threshold = 30
        # 동일한 댓글을 (Threshold)개 이상으로 포함하는 키워드는 서로 합치기
        groups = defaultdict(set)  # 각 키워드 그룹에 속할 댓글을 저장할 딕셔너리
        visited = set()  # 이미 합쳐진 키워드 트래킹
        for keyword1, indices1 in indices.items():
            # 키워드가 합쳐지지 않았다면 새로 그룹을 생성
            if keyword1 in visited:
                continue
            groups[keyword1].update(indices1)

            # 겹치는 댓글이 (Threshold)개 이상인지 확인하고 맞다면 키워드 합치기
            for keyword2, indices2 in indices.items():
                if keyword1 != keyword2:
                    overlap_count = len(indices1 & indices2)
                    if overlap_count >= overlap_threshold:
                        groups[keyword1].update(indices2)
                        visited.add(keyword2)
        # 그룹 형식 변환 ({'트럼프':{1,2,3},'바이든':{4,5,6}} => {'트럼프':(1,2,3),'바이든':(4,5,6)})
        final_groups = {}
        for keyword, group_indices in groups.items():
            group_indices = sorted(group_indices)  # Sort indices for consistency
            if tuple(group_indices) not in final_groups.values():
                final_groups[keyword] = tuple(group_indices)
        # 그룹 간 중복되는 댓글 제거
        assigned_indices = set()  # 이미 최종 그룹에 할당된 댓글 트래킹
        cleaned_groups = {} # 최종 그룹
        # 그룹 별 댓글 수를 기준으로 정렬
        sorted_groups = sorted(final_groups.items(), key=lambda x: len(x[1]), reverse=True)
        # 같은 댓글이 여러 개의 그룹에 중복되어 속해있는 경우, 가장 많은 댓글을 가진 그룹에 속한 댓글만 남겨두고 나머지는 제거
        for keyword, group_indices in sorted_groups:
            unique_indices = tuple(idx for idx in group_indices if idx not in assigned_indices)
            if unique_indices:  # Only keep groups with remaining indices
                cleaned_groups[keyword] = unique_indices
                assigned_indices.update(unique_indices)
        # 전체 그룹 중에서 댓글 수 기준으로 Top 5에 해당하는 그룹을 추출 (스팸 댓글들도 삭제되는 효과)
        top_cleaned_groups = sorted(cleaned_groups.items(), key=lambda x: len(x[1]), reverse=True)[:5]

        indices = set()
        for group, group_indices in top_cleaned_groups:
            indices.update(group_indices)

        final_keywords = [group[0] for group in top_cleaned_groups]
        filtered_df = self.df.loc[list(indices)].reset_index(drop=True)
        
        return filtered_df, final_keywords

## YoutubeClusterModel

- reduce_emb_dim: 댓글 embedding의 차원을 감소시킵니다.
- get_clusters: 댓글 clustering을 진행합니다.

In [95]:

class YoutubeClusterModel:
    def __init__(self, df):
        self.df = df
        self.reduced_embeddings = None
        
    def reduce_emb_dim(self):
        embeddings = self.df['embeddings'].tolist()

        reducer = UMAP(n_neighbors=10, n_components=20, min_dist = 0.1, metric='cosine', random_state=42)
        self.reduced_embeddings = reducer.fit_transform(embeddings)

    def get_clusters(self):
        num_clusters = 4 # 클러스터 개수 설정
        gmm = GaussianMixture(n_components=num_clusters, random_state=42)
        self.df['cluster'] = gmm.fit_predict(self.reduced_embeddings)  # GMM 학습 & 추론
        
        return self.df


## GPTSummarizer

- do_sample: 좋아요 기반으로 댓글들을 추출합니다.
- get_summary: open AI chat GPT에 추출한 댓글들의 요약을 요청합니다.

In [ ]:
class GPTSummarizer:
    def __init__(self, df, isClustered=True):
        self.isClustered = isClustered
        self.df = df
        if not self.isClustered:
            self.df['cluster'] = [0]*len(df)
        
    def do_sample(self):
        sampled_comments = []
        n_samples_per_cluster = 10 if self.isClustered else 10*4  # 샘플링 할 개수
        
        for cluster in self.df['cluster'].unique():
            cluster_df = self.df[self.df['cluster'] == cluster]
            sampled_comments.append(weighted_sample(cluster_df, n_samples_per_cluster))
            
        self.sampled_comments_df = pd.concat(sampled_comments)
        return self.sampled_comments_df
    
    def get_summary(self):
        clustered_dict = self.sampled_comments_df.groupby('cluster')['comment'].apply(list).to_dict()
        formatted_string = str(clustered_dict)

        # ChatGPT에게 댓글을 보고 전반적인 경향성 요약 요청
        openai.api_key = "" #API KEY

        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 유튜브 댓글에서 클러스터링을 통해 대표적으로 선별된 몇가지 댓글들을 보고 댓글들의 전체적인 경향성을 요약해서 나타내는 모델입니다."},
                {"role": "user", "content": "다음은 각 클러스터에 속하는 댓글을 가지고온 것입니다. 이것들에 기반해서 전체적인 댓글의 경향성을 요약해주세요."+
                "각 클러스터에 대해서 설명하는 것이 아닌, 문장이 자연스럽게 이어지도록 서술해주세요."
                "존댓말로 서술해주세요."+
                formatted_string}
            ]
        )

        summary = response.choices[0].message.content
        return summary

## Embedding 모델 및 불용어사전 선언

In [ ]:
model = SentenceTransformer("jhgan/ko-sroberta-multitask")
mecab = Mecab()
stopwords = ['은', '는', '이', '가', '을', '를', '에', '의', '로', '와', '과', '하다', '있다', '없다']

## Embedding 생성

In [ ]:
embedder = CommentEmbedder(model, mecab, stopwords)
embedder.get_comments(video_id)
embedder.eliminate_stopwords()
raw_df, word_embeddings_dict = embedder.get_embbedings()

# Experiment 1
Raw Comment + GPT Summary

In [ ]:
summarizer = GPTSummarizer(raw_df, isClustered=False)
summarizer.do_sample()
final_summary = summarizer.get_summary()
print(final_summary)

# Experiment 2
Raw Comment + Keyword Preprocessing + GPT Summary

In [ ]:
extractor = KeywordExtractor(raw_df, word_embeddings_dict)
extractor.get_keyword_groups()
filtered_df, final_keywords = extractor.extract_keyword_groups()

summarizer = GPTSummarizer(filtered_df, isClustered=False)
summarizer.do_sample()
final_summary = summarizer.get_summary()
print(final_summary)

댓글을 보면, 대부분의 관객들이 하늘 씨의 매력에 매료된 것을 알 수 있습니다. 그녀의 섹시하면서도 절제된 퍼포먼스, 무표정한 표정 변화, 그리고 '터미네이터'와 '매트릭스'와 같은 대중문화적 비유를 통해 그녀의 무대 매력을 칭찬하고 있습니다. 또한 스타일과 안무는 물론, 하늘 씨 개인의 매력이 무대 위에서 잘 발휘되었다는 평가를 받았습니다. 

곡과 관련해서, '더블 타이틀'이라는 말이 자주 등장하며 이를 통해 관객들이 새로운 컴백을 기대하는 모습을 확인할 수 있습니다. 굉장히 높은 기대치를 가지고 있음을 알 수 있습니다. 또한 그들의 노래와 패션에 대한 호평도 이어집니다.

무대 선정과 카메라 워크에 대해서도 긍정적인 평가가 많이 나왔습니다. 특히, 카메라 구도와 조명, 그리고 무대 위의 전환 등이 무대 퍼포먼스를 더욱 돋보이게 했다는 평가를 받았습니다.

그러나 동시에 일부 댓글에서는 무대 선정에 대한 아쉬움이나 노래 선택에 대한 다른 의견도 제시되었습니다. 그럼에도 불구하고 그들의 노래와 퍼포먼스에 대한 사랑과 열정이 더욱 돋보이는 것을 확인할 수 있었습니다. 

마지막으로, 팬들은 키 오프의 '스타일'과 '안무 장악력'에 대해 큰 감탄을 보이며, 섬세하고 완벽한 퍼포먼스를 칭찬했고, 그들이 다음에 어떤 무대를 선보일지 기대하고 있음을 보여주었습니다.


# Experiment 3
Raw Comment + Clustering + GPT Summary

In [ ]:
ytModel = YoutubeClusterModel(raw_df)
ytModel.reduce_emb_dim()
clustered_df = ytModel.get_clusters()

summarizer = GPTSummarizer(clustered_df, isClustered=True)
summarizer.do_sample()
final_summary = summarizer.get_summary()
print(final_summary)

/home/changoo/.local/share/virtualenvs/comment-6N9L2wjh/lib/python3.8/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


전체적으로 댓글들은 비디오의 이해를 위한 피드백이다 보니, 구체적인 부분을 지적하며 양질의 피드백이 주를 이룹니다. 특히 안무와 노래의 조화에 주목하며, 특정 댄서나 멤버를 지목하여 그들의 능력을 칭찬하는 내용으로 이루어져 있습니다. 또한, 이 중에서도 어떤 댓글은 본 영상의 특별한 요소나 충격적인 부분을 강조하며 강한 감탄을 표현하고 있습니다. 전체 곡의 분위기나 표현에 칭찬을 보내는 댓글들도 비교적 많이 보여, 시청자들이 영상에 대해 만족해하며 좋은 반응을 보이고 있다는 것을 알 수 있습니다. 시청자들은 또한, 개인적으로 가장 인상적이었던 파트를 언급하며 그 부분을 특별히 선호한다는 점을 강조하고 있습니다. 이런 것들을 종합해 보면, 이들 댓글들은 전반적으로 영상에 대한 만족도가 높으며, 갖가지 디테일에 대한 적극적인 공감과 인상적인 부분에 대한 자세한 피드백을 통해 높은 관심과 호평을 보이고 있다는 것을 알 수 있습니다.


# Experiment 4
Raw Comment + Keyword Preprocessing + Clustering + GPT Summary

In [ ]:
extractor = KeywordExtractor(raw_df, word_embeddings_dict)
extractor.get_keyword_groups()
filtered_df, final_keywords = extractor.extract_keyword_groups()

ytModel = YoutubeClusterModel(filtered_df)
ytModel.reduce_emb_dim()
filtered_clustered_df = ytModel.get_clusters()

summarizer = GPTSummarizer(filtered_clustered_df, isClustered=True)
summarizer.do_sample()
final_summary = summarizer.get_summary()
print(final_summary)

/home/changoo/.local/share/virtualenvs/comment-6N9L2wjh/lib/python3.8/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


댓글들을 살펴본 결과, 전반적으로 노래와 안무에 대한 큰 마음에 찬 사랑과 함께 특히 '키 오프' 노래에 대한 높은 관심을 확인할 수 있습니다. 참고로 키 오프는 인기를 끌었던 노래 중 하나로 보입니다. 더불어 몇몇 댓글러들은 또다른 노래를 '타이틀 곡'으로 제안하는 경향을 보였습니다. 특정 멤버, '하늘'에 대한 관심도 두드러지게 나타났는데, 그의 표정 변화나 컨셉에 대한 칭찬이 이어졌습니다. 일부 댓글러들은 '이거'라는 표현을 주로 사용해 특정 내용에 대한 강한 호감이나 인상을 나타냈습니다. 이런 코멘트들을 통해 댓글러들이 노래와 안무에 대한 만족감을 표현하며 동시에 특정 멤버에게 강한 호감을 표현하는 경향을 보입니다.
